### 1. Download Dataset

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, ConfusionMatrixDisplay
)

### 2. Load Dataset

In [3]:
PATH = "train.csv"  # change if needed
df = pd.read_csv(PATH)

# Set column names here if your file differs
TEXT_COL = "tweet"
LABEL_COL = "label"

# Basic sanity checks
if TEXT_COL not in df.columns or LABEL_COL not in df.columns:
    raise ValueError(
        f"Expected columns '{TEXT_COL}' and '{LABEL_COL}'. "
        f"Found: {list(df.columns)}. Update TEXT_COL/LABEL_COL accordingly."
    )